In [41]:
from src.conv_onet.models.decoder_copy import NICE
import torch
import torch.nn as nn
import torch.nn.functional as F

In [42]:
grid_coarse = torch.zeros([1, 32, 3, 4, 6])
grid_middle = torch.zeros([1, 32, 10, 12, 20])
grid_fine = torch.zeros([1, 32, 21, 25, 41])
grid_color = torch.zeros([1, 32, 21, 25, 41])

In [43]:
bound = torch.Tensor([[0.0,6.5],[0.0,4.0],[0,3.5]])

In [5]:
coarse_len = 2
middle_len = 0.32
fine_len = 0.16

In [44]:
c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
               "grid_fine": grid_fine, "grid_color": grid_color})

In [45]:
model = NICE(coarse = True)

In [111]:
x_range = torch.arange(bound[0][0], bound[0][1], middle_len)
y_range = torch.arange(bound[1][0], bound[1][1], middle_len)
z_range = torch.arange(bound[2][0], bound[2][1], middle_len)

In [7]:
p_list = []
for i in x_range:
    for j in y_range:
        for k in z_range:
            p_list.append([i, j, k])

p_tensor = torch.Tensor(p_list)
model.forward(p_tensor, c_grid, 'coarse')[:, -1]


NameError: name 'x_range' is not defined

In [8]:
p = torch.Tensor([[0,0,0], [0,0,0]])

In [59]:
feature_0 = torch.arange(0, 10, 0.1)
test_prob = []
for fea in feature_0:
    grid_middle = torch.zeros([1, 32, 10, 12, 20])
    grid_middle[0, 1, 0, 0, 0] = fea
    c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
                "grid_fine": grid_fine, "grid_color": grid_color})
    prob = model.forward(p, c_grid, 'middle')[:, -1][0]
    if prob >-0.1 and prob <0.1:
        test_prob.append([grid_middle[0, 1, 0, 0, 0], prob])

In [20]:
test_prob

[]

In [36]:
import numpy as np

In [93]:
from torch.autograd import Variable
p = torch.Tensor([[0,0,0], [0,1,0]])
middle_grid_para = Variable(torch.zeros(32), requires_grad = True)
optimizer = torch.optim.Adam([{'params': middle_grid_para, 'lr': 0.01}])
loss1 = torch.Tensor([3]).pow(2).sum()
for n in range(50):

    optimizer.zero_grad()
    grid_middle = torch.zeros([1, 32, 10, 12, 20])
    grid_middle[0, :, 0, 0, 0] = middle_grid_para
    c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
               "grid_fine": grid_fine, "grid_color": grid_color})
    prob = model.forward(p, c_grid, 'middle')[:, -1][0]
    loss = (prob).pow(2).sum()
    print(loss)
    if n>1 and loss.item()<loss1.item():
        print(1)
        final_middle = middle_grid_para
        loss1 = loss
    loss.backward()
    optimizer.step()


tensor(0.0106, grad_fn=<SumBackward0>)
tensor(0.0008, grad_fn=<SumBackward0>)
tensor(0.0011, grad_fn=<SumBackward0>)
tensor(0.0037, grad_fn=<SumBackward0>)
tensor(0.0038, grad_fn=<SumBackward0>)
tensor(0.0019, grad_fn=<SumBackward0>)
1
tensor(0.0003, grad_fn=<SumBackward0>)
1
tensor(9.2323e-05, grad_fn=<SumBackward0>)
1
tensor(0.0009, grad_fn=<SumBackward0>)
tensor(0.0015, grad_fn=<SumBackward0>)
tensor(0.0015, grad_fn=<SumBackward0>)
1
tensor(0.0008, grad_fn=<SumBackward0>)
1
tensor(0.0001, grad_fn=<SumBackward0>)
1
tensor(2.6694e-05, grad_fn=<SumBackward0>)
1
tensor(0.0004, grad_fn=<SumBackward0>)
tensor(0.0007, grad_fn=<SumBackward0>)
tensor(0.0008, grad_fn=<SumBackward0>)
tensor(0.0005, grad_fn=<SumBackward0>)
1
tensor(0.0002, grad_fn=<SumBackward0>)
1
tensor(3.8124e-07, grad_fn=<SumBackward0>)
1
tensor(0.0001, grad_fn=<SumBackward0>)
tensor(0.0003, grad_fn=<SumBackward0>)
tensor(0.0004, grad_fn=<SumBackward0>)
tensor(0.0003, grad_fn=<SumBackward0>)
1
tensor(8.4564e-05, grad_fn=<Su

KeyboardInterrupt: 

In [90]:

final_middle

tensor([ 0.0312, -0.0013,  0.0069,  0.0369, -0.0710,  0.0380, -0.0020,  0.0252,
        -0.0129,  0.0283, -0.0256,  0.0732, -0.0091, -0.0298, -0.0052,  0.0092,
         0.0329,  0.0244,  0.0750,  0.1114, -0.0270, -0.0052,  0.0126, -0.0081,
         0.0037, -0.0259,  0.0342,  0.0333,  0.0150, -0.0820,  0.0344,  0.0101],
       requires_grad=True)

In [91]:
grid_middle = torch.zeros([1, 32, 10, 12, 20])
grid_middle[0, :, 0, 0, 0] = final_middle
c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
            "grid_fine": grid_fine, "grid_color": grid_color})
model.forward(p, c_grid, 'middle')[:, -1][0]

tensor(0.0042, grad_fn=<SelectBackward>)